# **Getting started with TensorFlow**

**Learning objectives**

1. Practice defining and performing basic operations on tensors
2. Use TensorFlow automatic differentiation capability
3. Learn how to train a linear regression from scratch with TensorFlow

In this notebook, we will start by reviewing the main operations on tensors in TensorFlow and understand how to manipulate TensorFlow Variables. We explain how these are compatible with Python built-in `list` and NumPy `array` objects.

Then we will jump to the problem of training a linear regression from scratch with gradient descent. The first order of business will be to understand how to compute the gradients of a function (a loss function here) with respect to some of its arguments (the model weights here). The TensorFlow construct allowing us to do that is `tf.GradientTape`, which we will describe.

At last we will create a simple training loop to learn the weights of a 1-dim linear regression using synthetic data generated from a linear model.

As a bonus exercise, we will do the same for data generated from a non-linear model, forcing us to manual-engineer non-linear features to improve our linear model performance.

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf

print(tf.__version__)

2.4.1


## **Operations on tensors**

### **Variables and constants**

Tensors in TensorFlow are either **constants** (`tf.constant`) or **variables** (`tf.Variable`). Constant values cannot be changed, while variable values can be.

The main difference is that instances of `tf.Variable` have methods allowing us to change their values while tensors constructed with `tf.constant` don't have these methods, and therefore their values cannot be changed. When you want to change the value of a `tf.Variable` `x` use one of the following methods:

- `x.assign(new_value)`
- `x.assign_add(value_to_be_added)`
- `x.assign_sub(value_to_be_subtracted)`

In [2]:
x = tf.constant([2, 3, 4])
x

<tf.Tensor: shape=(3,), dtype=int32, numpy=array([2, 3, 4], dtype=int32)>

In [3]:
x = tf.Variable(2.0, dtype=tf.float32, name="my_variable")

In [4]:
x.assign(45.8)
x

<tf.Variable 'my_variable:0' shape=() dtype=float32, numpy=45.8>

In [5]:
x.assign_add(4)
x

<tf.Variable 'my_variable:0' shape=() dtype=float32, numpy=49.8>

In [6]:
x.assign_sub(49.8)
x

<tf.Variable 'my_variable:0' shape=() dtype=float32, numpy=0.0>

### **Point-wise operations**

TensorFlow offers similar point-wise tensor operations as NumPy does:

- `tf.add` allows to add the components of a tensor
- `tf.multiply` allows to multiply the components of a tensor
- `tf.subtract` allows to subtract the components of a tensor
- `tf.math.*` contains the usual `math` operations to be applied on the components of a tensor

Most of the standard arithmetic ops (`tf.add`, `tf.subtract`, etc.) are overloaded by the usual corresponding arithmetic symbols (`+`, `-`, `*`, etc.)

In [8]:
a = tf.constant([5, 3, 8])
b = tf.constant([3, -1, 2])
c = tf.add(a, b)
d = a + b

print("c:", c)
print("d:", d)

c: tf.Tensor([ 8  2 10], shape=(3,), dtype=int32)
d: tf.Tensor([ 8  2 10], shape=(3,), dtype=int32)


In [9]:
a = tf.constant([5, 3, 8])
b = tf.constant([3, -1, 2])
c = tf.multiply(a, b)
d = a * b

print("c:", c)
print("d:", d)

c: tf.Tensor([15 -3 16], shape=(3,), dtype=int32)
d: tf.Tensor([15 -3 16], shape=(3,), dtype=int32)


In [10]:
# tf.math.exp expects floats so we need to explicitely give the type
a = tf.constant([5, 3, 8], dtype=tf.float32)
b = tf.math.exp(a)

print("b:", b)

b: tf.Tensor([ 148.41316    20.085537 2980.958   ], shape=(3,), dtype=float32)


### **Point-wise operations**

In addition to naive TensorFlow tensors, tensorflow operations can take native Python types and NumPy `array` operands.

In [13]:
# Native Python lists
a_py = [1, 2]
b_py = [3, 4]

tf.add(a_py, b_py)

<tf.Tensor: shape=(2,), dtype=int32, numpy=array([4, 6], dtype=int32)>

In [14]:
# NumPy arrays
a_np = np.array([1, 2])
b_np = np.array([3, 4])

tf.add(a_np, b_np)

<tf.Tensor: shape=(2,), dtype=int64, numpy=array([4, 6])>

In [15]:
# Native TensorFlow tensors
a_tf = tf.constant([1, 2])
b_tf = tf.constant([3, 4])

tf.add(a_tf, b_tf)

<tf.Tensor: shape=(2,), dtype=int32, numpy=array([4, 6], dtype=int32)>

You can convert a native TensorFlow tensor to a NumPy array using `.numpy()`.